In [ ]:
#1. Importações e Configuração
import pandas as pd
import os
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score
from shapash import SmartExplainer

#Configuração dos caminhos:
base_path = os.getcwd()

if os.path.basename(base_path) == 'notebooks':
    data_path = os.path.abspath(os.path.join(base_path, '..', 'data', 'processed', 'hydraulic_uci_processed.csv'))
    docs_path = os.path.abspath(os.path.join(base_path, '..', 'docs', 'hydraulic'))
else:
    data_path = os.path.abspath(os.path.join(base_path, 'data', 'processed', 'hydraulic_uci_processed.csv'))
    docs_path = os.path.abspath(os.path.join(base_path, 'docs', 'hydraulic'))

os.makedirs(docs_path, exist_ok=True)
print(f"-> Dados serão lidos de: {data_path}")
print(f"-> Relatórios serão salvos em: {docs_path}")

-> Dados serão lidos de: c:\XAILabProcess\data\processed\hydraulic_uci_processed.csv
-> Relatórios serão salvos em: c:\XAILabProcess\docs\hydraulic


In [7]:
#2. Carga de Dados
try:
    df = pd.read_csv(data_path)
except FileNotFoundError:
    print("\nERRO CRÍTICO: Arquivo 'hydraulic_uci_processed.csv' não encontrado.")
    print("Execute o notebook '05_etl_processamento_hidraulico.ipynb' primeiro para gerar este arquivo.")
    raise

#Definição de X (Sensores) e y (Target: 0=Estável, 1=Instável)
y = df['Stable_Flag']
X = df.drop('Stable_Flag', axis=1)

#Divisão Treino/Teste (70/30)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

In [8]:
#3. Treinamento do Modelo (Auditoria):

print("Treinando modelo Random Forest (pode levar alguns segundos)...")
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)
print("Modelo treinado com sucesso.")

Treinando modelo Random Forest (pode levar alguns segundos)...
Modelo treinado com sucesso.


In [9]:
#4. Configuração para XAI:
#Dicionário:
features_dict = {
    'Pressure_1': 'Pressão Principal (bar)',
    'Pressure_2': 'Pressão Secundária (bar)',
    'Pressure_3': 'Pressão de Retorno (bar)',
    'Temperature_1': 'Temp. Tanque (°C)',
    'Temperature_2': 'Temp. Motor (°C)',
    'Temperature_3': 'Temp. Trocador (°C)',
    'Vibration': 'Vibração (mm/s)',
    'Volume_Flow_1': 'Fluxo Principal (L/min)',
    'Volume_Flow_2': 'Fluxo Secundário (L/min)',
    'Cooling_Efficiency': 'Eficiência do Resfriador (%)',
    'Cooling_Power': 'Potência de Resfriamento (kW)',
    'Efficiency_Factor': 'Fator de Eficiência'
}

xpl = SmartExplainer(
    model=model,
    features_dict=features_dict
)

xpl.compile(x=X_test, y_target=y_test)
print("Explicações compiladas.")

INFO: Shap explainer type - <shap.explainers._tree.TreeExplainer object at 0x0000025747B37CD0>
Explicações compiladas.


In [ ]:
#5. Geração dos artefatos (Outputs)
print("\nGerando arquivos de evidência...")

#Artefato A: Importância Global (Quais sensores mais impactam a estabilidade?)
file_global = os.path.join(docs_path, "1_Hydraulic_Feature_Importance.html")
xpl.plot.features_importance().write_html(file_global)
print(f" [OK] Gráfico Global salvo: {file_global}")

#Artefato B: Explicação Local (Por que o sistema ficou instável neste caso?)
#Busca o primeiro caso instável (Target=1) para exemplificar
casos_instaveis = y_test[y_test == 1].index
id_caso = casos_instaveis[0] if len(casos_instaveis) > 0 else X_test.index[0]

file_local = os.path.join(docs_path, f"2_Hydraulic_Caso_Local_ID{id_caso}.html")
xpl.plot.local_plot(index=id_caso).write_html(file_local)
print(f" [OK] Gráfico Local salvo (ID {id_caso}): {file_local}")

#Artefato C: Métricas de Performance
acc = accuracy_score(y_test, model.predict(X_test))
f1 = f1_score(y_test, model.predict(X_test))

texto_metricas = f"""
RELATÓRIO DE AUDITORIA - SISTEMAS HIDRÁULICOS (UCI)
===================================================
Data: 07/02/2026
Dataset: Condition Monitoring of Hydraulic Systems (Real)
Modelo: Random Forest Classifier

METRICAS DE VALIDAÇÃO:
----------------------
Acurácia Global: {acc:.4f}
F1-Score (Classe Instável): {f1:.4f}

OBSERVAÇÕES:
Dados reais de sensores físicos agregados por ciclo de trabalho (60s).
"""

file_metrics = os.path.join(docs_path, "3_Hydraulic_Metricas.txt")
with open(file_metrics, "w", encoding="utf-8") as f:
    f.write(texto_metricas)
print(f" [OK] Relatório de Texto salvo: {file_metrics}")

print("\nSUCESSO! Todos os documentos de auditoria foram gerados na pasta 'docs/hydraulic'.")


Gerando arquivos de evidência...
 [OK] Gráfico Global salvo: c:\XAILabProcess\docs\hydraulic\1_Hydraulic_Feature_Importance.html
 [OK] Gráfico Local salvo (ID 119): c:\XAILabProcess\docs\hydraulic\2_Hydraulic_Caso_Local_ID119.html
 [OK] Relatório de Texto salvo: c:\XAILabProcess\docs\hydraulic\3_Hydraulic_Metricas.txt

SUCESSO! Todos os documentos de auditoria foram gerados na pasta 'docs/hydraulic'.
